# CUNEF
### Trabajo de Final de Master 

# 03. Modelado y Análisis de resultados

In [ ]:
# Importamos las librerias que van a ser utilizados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pickle

import stellargraph as sg
from stellargraph.layer import GCN_LSTM

from sklearn.metrics import mean_squared_error,mean_absolute_error 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import  Model
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from keras.utils.vis_utils import plot_model

In [ ]:
# Funcion definida para realizar la ventana de los datos con los que se van a trabajar

def sequence_data_preparation_predict(seq_len, pre_len, test_data_pred):
    testX_pred = []

    for i in range(test_data_pred.shape[1] - int(seq_len + pre_len - 1)):
        b = test_data_pred[:, i : i + seq_len + pre_len]
        testX_pred.append(b[:, :seq_len])

    testX_pred = np.array(testX_pred)

    return testX_pred

def extract_metrics_from_predicted(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred).round(2)
    rmse = np.sqrt(mse).round(2)
    mae = mean_absolute_error(y_true, y_pred).round(2)
    mape =np.mean(np.abs((y_true - y_pred) / y_true)* 100).round(2)
    return (f'mse :{mse}, rmse: {rmse}, mae: {mae}, mape: {mape}')


### 1. Importamos los datos a trabajar

Se importan los datos tratados previamente en pasos anteriores

In [ ]:
# Datos temporales sobre intensidad de tráfico
trainX=np.load('../datos/03_procesados/trainX.npy')
trainY=np.load('../datos/03_procesados/trainY.npy')
valX=np.load('../datos/03_procesados/valX.npy')
valY=np.load('../datos/03_procesados/valY.npy')
testX=np.load('../datos/03_procesados/testX.npy')
testY=np.load('../datos/03_procesados/testY.npy')
val_11=np.load('../datos/03_procesados/pred_11.npy')

test_scaled=np.load('../datos/03_procesados/test_scaled.npy')

# Datos Espaciales sobre la localización de sensores
matrix_lat_long = pickle.load(open('../datos/04_pickles/matrix_lat_long_final', 'rb'))

### 2. Creamos el modelo de Predicción

Aqui se crea la arquitectura del modelo de predicción utilizado en base a la información proporcionada por la librería StellarGraph

In [ ]:
gcn_lstm = GCN_LSTM(
    seq_len=10,
    adj=matrix_lat_long,
    gc_layer_sizes=[16, 10],
    gc_activations=["relu", "relu"],
    lstm_layer_sizes=[200,200],
    lstm_activations=["tanh", "tanh"],
    dropout=0.1
)

In [ ]:
#Builds a GCN model for node  feature prediction
x_input, x_output = gcn_lstm.in_out_tensors()

model = Model(inputs=x_input, outputs=x_output)

In [ ]:
# Seleccionamos el optimizador a utilizar y el ratio de aprendizaje
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="mae", metrics=["mse"])

In [ ]:
# Entrenamos el modelo creado con los datos de train y verificamos la funcion de perdida con los datos de validacion
history = model.fit(
    trainX,
    trainY,
    epochs=200,
    batch_size=60,
    shuffle=True,
    verbose=1,
    validation_data=(valX, valY),
)

model.summary()

Verificamos la arquitectura de la red neuronal realizada

In [ ]:
plt.savefig('model_plot.png')
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
print(
    "Train loss: ",
    history.history["loss"][-1]
)

In [ ]:
# Gráfico de función de pérdida
sg.utils.plot_history(history)

### 3.Creación de predicciones

Primera prediccion de valores

La primera prediccion se realiza tomando como entrada los 10 valores separados en el dataset de testX

In [ ]:
# Realizamos la primera predicción del modelo

train_pred = model.predict(trainX)
pred_1= model.predict(testX)    

### Segunda prediccion de valores

Para la segunda predicción de 30 min tomamos el valor de la primera predicción realizada y agregamos a los datos de entrenamiento para con los 10 valores (9 valores pasados y 1 valor de la predicción 1) predecimos los siguientes 30 min

In [ ]:
# Copiamos el dataset escalado original
test_data_2=test_scaled.copy()
test_data_2=pd.DataFrame(test_data_2)
val_10=test_data_2.iloc[:,-1] # nos quedamos con el valor 10 de la serie de textX
test_data_2=test_data_2.drop([0,10],axis=1)

# Introducimos el valor predicho anteriormente
list_pred=pd.DataFrame(pred_1)
test_data_2['pred_10']=list_pred.T
test_data_2['pred_data']=0

# Pasamos los datos a array, formato que trabaja el modelo
test_data_pred = np.array(test_data_2)

# Colocamos los datos con la ventana de observaciones con las que vamos a trabajar
testX_pred = sequence_data_preparation_predict(10, 1, test_data_pred)
testX_pred.shape

Realizamos la prediccion con un nuevo dataset de 10 valores

In [ ]:
pred_2=  model.predict(testX_pred)

In [ ]:
# La nueva predicción la guardamos como un dataframe
list_pred=pd.DataFrame(pred_2)

# Hacemos una copia del dataframe pasado y agregamos la segunda predicción realizada
df_pred=test_data_2.copy()
df_pred['pred_11']=list_pred.T


### 4. Viasualizamos valores reales y valores de resultado en las predicciones

In [ ]:
# Insertamos los valores reales de las predicciones 1 y 2 al nuevo dataframe
df_pred[10]=val_10
df_pred[11]=val_11

# Ordenamos las columnas del nuevo dataframe
col_order=[1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,'pred_10','pred_11']
df_pred=df_pred[col_order]
df_pred

Los valores normalizados los re-escalamos para trabajar con el rango inicial de intensidad de tráfico

In [ ]:
# Identificamos valor minimo y máximo definido para el proceso de re-escalado
max_speed = 10715.0
min_speed= 0

# Reescalamos la tabla resultante de valores real vs predicciones
df_pred = df_pred * (max_speed - min_speed) + min_speed
df_pred

### 5. Métricas de salida de valores predichos por red neuronal

In [ ]:
# Seleccionamos las variables con las vamos a comparar las predicciones
y_true_0=df_pred.loc[:,10]
y_true_1=df_pred.loc[:,11]

y_pred_0=df_pred.loc[:,'pred_10']
y_pred_1=df_pred.loc[:,'pred_11']

In [ ]:
print(f'Salida de la segunda prediccion {extract_metrics_from_predicted(y_true_0, y_pred_0)}')
print(f'Salida de la primera prediccion {extract_metrics_from_predicted(y_true_1, y_pred_1)}')

### 6. Comparando el modelo de redes neuronales utilizando un modelo base

Creamos un modelo base el cual tome el valor anterior de la serie temporal com predicción

In [ ]:
# Importamos todos los datos de intensidad de sensores
pred_speed_data=pd.read_csv("../datos/03_procesados/datos_procesados.csv", index_col='id')
pred_speed_data['-1']=pred_speed_data.iloc[:,0]

# Cambio el orden de las columnas para que el primer dato (columna -1) quede en la parte inicial del dataframe
cols = pred_speed_data.columns.tolist()
cols = cols[-1:] + cols[:-1]


base_model=pred_speed_data[cols].iloc[:,:-1]
base_model=base_model.reset_index()
b_m=base_model.drop(['id'], axis=1)
b_m

In [ ]:
bs_pred_0=b_m.iloc[:,-2]
bs_pred_1=b_m.iloc[:,-1]

In [ ]:
print(f'Salida de la primera prediccion {extract_metrics_from_predicted(y_true_0, bs_pred_0)}')
print(f'Salida de la segunda prediccion {extract_metrics_from_predicted(y_true_1, bs_pred_1)}')

## Visualización de comportamiento de modelos

Seran seleccionados un conjunto de sensores apra ver su comportamiento en relacion a los datos reales, el modelo base creado y el modelo T-GCN con ffin de evaluar su desempeño.

In [ ]:
# Se seleccionan los IDs a evaluar
list_id=[4009, 4005, 4011, 4024, 3994, 10100]

Creamos los dataframes los que serán trabajados

In [ ]:
# Importo los datos reales con los que vamos a trabajar
real_caso=pd.read_csv("../datos/03_procesados/datos_procesados.csv")
real_caso=real_caso[real_caso['id'].isin(list_id)].set_index(['id'])
real_caso=real_caso.iloc[:,10:9398]

# Coloco el modelo base con las condiciones para poder realizar la comparacion, seleccionando los sensores a evaluar
bm_caso=base_model[base_model['id'].isin(list_id)].set_index(['id'])
bm_caso=bm_caso.iloc[:,10:9398]

# Trabajamos los datos de la salida del modelo NN para realizar las comparaciones
max_speed = 10715
pre_model=pd.DataFrame(train_pred)

pre_model=(pre_model*max_speed).T #multiplicamos por el valor maximo de los datos del modelo y buscamos su transpuesta
pre_model['id']=base_model['id'] # Los datos no presentan el id del sensor, se busca para ingresarlos
pm_caso=pre_model[pre_model['id'].isin(list_id)]
pm_caso=pm_caso.set_index(['id'])

### Verificamos las gráficas de salida para cada modelo creado

Gráfico para sensor 4005 en posición 2 del dataframe

In [ ]:
# Se selcciona el intervalo a analizar
indices = range(1, 200)

# Se crea el gráfico para los datos reales y la comparación con los modelos

plt.figure(figsize=(10, 3))
plt.plot(real_caso.columns[indices], real_caso.iloc[2, indices], color='red', label='Real')
plt.plot(real_caso.columns[indices], bm_caso.iloc[2, indices], label='Modelo base')
plt.plot(real_caso.columns[indices], pm_caso.iloc[2, indices], color='green', label='Modelo NN')

multiples_of_10 = np.arange(10, 200, 10)

plt.xticks(real_caso.columns[multiples_of_10])  # Establecer marcas solo en múltiplos de 10
plt.legend()
plt.title(f'Gráfico comparativo resultados de modelos de predicción tráfico para sensor 4005')
plt.show()

Gráfico para sensor 4009 en posición 3 del dataframe

In [ ]:
# Se selcciona el intervalo a analizar
indices = range(1, 200)

# Se crea el gráfico para los datos reales y la comparación con los modelos

plt.figure(figsize=(10, 3))
plt.plot(real_caso.columns[indices], real_caso.iloc[3, indices], color='red', label='Real')
plt.plot(real_caso.columns[indices], bm_caso.iloc[3, indices], label='Modelo base')
plt.plot(real_caso.columns[indices], pm_caso.iloc[3, indices], color='green', label='Modelo NN')

multiples_of_10 = np.arange(10, 200, 10)

plt.xticks(real_caso.columns[multiples_of_10])  # Establecer marcas solo en múltiplos de 10
plt.title(f'Gráfico comparativo resultados de modelos de predicción tráfico para sensor 4009')

plt.legend()
plt.show()

Gráfico para sensor 10100 en posición 5 del dataframe

In [ ]:
# Se selcciona el intervalo a analizar
indices = range(1, 200)

# Se crea el gráfico para los datos reales y la comparación con los modelos

plt.figure(figsize=(10, 3))
plt.plot(real_caso.columns[indices], real_caso.iloc[5, indices], color='red', label='Real')
plt.plot(real_caso.columns[indices], bm_caso.iloc[5, indices], label='Modelo base')
plt.plot(real_caso.columns[indices], pm_caso.iloc[5, indices], color='green', label='Modelo NN')

multiples_of_10 = np.arange(10, 200, 10)

plt.xticks(real_caso.columns[multiples_of_10])  # Establecer marcas solo en múltiplos de 10
plt.title(f'Gráfico comparativo resultados de modelos de predicción tráfico para sensor 10100')

plt.legend()
plt.show()